In [1]:
import duckdb
import requests
from pathlib import Path
import gzip
import shutil

In [2]:
DATA_DIR = Path("./data") 

In [3]:
aggregate_data = False # @param ["False", "True"] {type:"raw"}

In [4]:
DATA_DIR = Path("../data")

DB_PATH = "ecommerce.duckdb"

# Connect to DuckDB (this creates a persistent file on disk)
con = duckdb.connect(DB_PATH)
if aggregate_data:

    MONTHS = [
    "2019-Oct", "2019-Nov", "2019-Dec",
    "2020-Jan", "2020-Feb", "2020-Mar", "2020-Apr"
    ]

    BASE_URL = "https://data.rees46.com/datasets/marketplace/"
    TMP_DIR = Path("tmp_downloads")
    TMP_DIR.mkdir(exist_ok=True)

    con.execute("DROP TABLE IF EXISTS session_summary")
    con.execute("""
    CREATE TABLE session_summary (
        user_id VARCHAR,
        user_session VARCHAR,
        product_id VARCHAR,
        brand VARCHAR,
        category_id BIGINT,
        category_code VARCHAR,
        price DOUBLE,
        view_time TIMESTAMP,
        added_to_cart BOOLEAN,
        purchased BOOLEAN
    )
    """)

    # Template SQL
    QUERY_TEMPLATE = """
    WITH base AS (
        SELECT *
        FROM '{file_path}'
        WHERE user_id IS NOT NULL AND product_id IS NOT NULL AND user_session IS NOT NULL
    ),
    view_events AS (
        SELECT
            user_id,
            user_session,
            product_id,
            MIN(event_time) AS view_time
        FROM base
        WHERE event_type = 'view'
        GROUP BY user_id, user_session, product_id
    ),
    flags AS (
        SELECT
            user_id,
            user_session,
            product_id,
            MAX(event_type = 'cart')::BOOLEAN AS added_to_cart,
            MAX(event_type = 'purchase')::BOOLEAN AS purchased,
            MAX(brand) AS brand,
            MAX(category_id) AS category_id,
            MAX(category_code) AS category_code,
            MAX(price) AS price
        FROM base
        GROUP BY user_id, user_session, product_id
    )
    SELECT
        f.user_id,
        f.user_session,
        f.product_id,
        f.brand,
        f.category_id,
        f.category_code,
        f.price,
        v.view_time,
        f.added_to_cart,
        f.purchased
    FROM flags f
    LEFT JOIN view_events v
    ON f.user_id = v.user_id AND f.user_session = v.user_session AND f.product_id = v.product_id
    """

    # Main loop
    for month in MONTHS:
        print(f"🔽 Downloading {month}...")
        url = f"{BASE_URL}{month}.csv.gz"
        gz_path = TMP_DIR / f"{month}.csv.gz"

        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(gz_path, 'wb') as f:
                shutil.copyfileobj(r.raw, f)

        print(f"📊 Processing {gz_path.name} into DuckDB without manual decompression...")
        query = QUERY_TEMPLATE.format(file_path=str(gz_path))
        con.execute(f"INSERT INTO session_summary {query}")

        gz_path.unlink()
        print(f"✅ {month} done and cleaned up.\n")

    print("🎉 All months processed. DuckDB file ready: ecommerce.duckdb")


In [5]:
result = con.execute("""
    SELECT *
    FROM session_summary
    WHERE purchased = TRUE
    LIMIT 10
""").df()
result

,user_id,user_session,product_id,brand,category_id,category_code,price,view_time,added_to_cart,purchased
0,554834118,3ccb47f7-1aea-4dc5-ae20-ac602b20737b,1004873,samsung,2053013555631882655,electronics.smartphone,388.81,2019-10-01 04:53:41,True,True
1,554235906,cc754ba3-8df5-44db-b740-bd963e309965,1004250,apple,2053013555631882655,electronics.smartphone,804.21,2019-10-01 04:56:42,False,True
2,543953063,e1dadb4f-d0e5-4ab5-b61b-c6ca8261d4ea,1801690,samsung,2053013554415534427,electronics.video.tv,368.04,2019-10-01 04:56:51,True,True
3,534837825,23543623-0a2e-b4b0-e339-ba30bff051aa,1004767,samsung,2053013555631882655,electronics.smartphone,254.76,2019-10-01 04:59:54,True,True
4,531172496,4ccfd0a1-5aa1-47e4-b2de-8fbcef83390a,1004767,samsung,2053013555631882655,electronics.smartphone,254.76,2019-10-01 05:14:27,True,True
5,512836766,667310e7-b0e7-4b5e-a4a6-770f15f84547,1004836,samsung,2053013555631882655,electronics.smartphone,241.18,2019-10-01 05:23:24,True,True
6,514914742,dfd392f2-1690-4229-91bc-229cc692d3cb,10700827,ubisoft,2053013561277416167,None,48.88,2019-10-01 05:27:02,False,True
7,512403922,d360fb2a-cc4c-4d49-a7c0-4b1fb36b2d7d,1002547,apple,2053013555631882655,electronics.smartphone,470.77,2019-10-01 05:29:34,False,True
8,515037132,c4aaa99c-81e7-4c9f-999b-ebb6b550bea5,4804055,apple,2053013554658804075,electronics.audio.headphone,188.73,2019-10-01 05:29:46,True,True
9,515070788,10deb55f-faf9-4a3a-a9df-ce134704c5ed,26025820,None,2053013562812531517,None,2.45,2019-10-01 05:31:18,False,True


In [8]:
query_top_view_pairs = """
SELECT 
    LEAST(a.category_code, b.category_code) AS cat1,
    GREATEST(a.category_code, b.category_code) AS cat2,
    COUNT(*) AS count
FROM session_summary a
JOIN session_summary b
  ON a.user_session = b.user_session
  AND a.category_code IS NOT NULL
  AND b.category_code IS NOT NULL
  AND a.category_code <> b.category_code
  AND a.product_id < b.product_id
GROUP BY cat1, cat2
ORDER BY count DESC
LIMIT 15
"""
top_view_pairs = con.execute(query_top_view_pairs).fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
top_view_pairs 

,cat1,cat2,count
0,appliances.kitchen.refrigerators,construction.tools.light,5277368
1,construction.tools.light,electronics.smartphone,5160877
2,computers.peripherals.printer,construction.components.faucet,3718100
3,construction.tools.light,electronics.clocks,3640765
4,apparel.shoes.sandals,appliances.kitchen.refrigerators,3496882
5,apparel.shoes,apparel.shoes.keds,3357421
6,computers.notebook,electronics.audio.headphone,3248338
7,construction.tools.light,sport.bicycle,2852870
8,auto.accessories.compressor,computers.peripherals.printer,2591912
9,construction.tools.light,electronics.camera.video,2581338


In [10]:
query_top_purchase_pairs = """
SELECT 
    LEAST(a.category_code, b.category_code) AS cat1,
    GREATEST(a.category_code, b.category_code) AS cat2,
    COUNT(*) AS count
FROM session_summary a
JOIN session_summary b
  ON a.user_session = b.user_session
  AND a.category_code IS NOT NULL
  AND b.category_code IS NOT NULL
  AND a.category_code <> b.category_code
  AND a.product_id < b.product_id
WHERE a.purchased = TRUE AND b.purchased = TRUE 
GROUP BY cat1, cat2
ORDER BY count DESC
LIMIT 15
"""
top_purchase_pairs = con.execute(query_top_purchase_pairs).fetchdf()
top_purchase_pairs

,cat1,cat2,count
0,construction.tools.light,sport.bicycle,17467
1,appliances.personal.massager,construction.tools.light,10706
2,appliances.kitchen.refrigerators,construction.tools.light,10672
3,construction.tools.light,electronics.clocks,9313
4,construction.tools.light,electronics.audio.headphone,8828
5,apparel.shoes,construction.tools.light,8031
6,electronics.audio.headphone,electronics.smartphone,4149
7,construction.tools.light,electronics.smartphone,4021
8,appliances.kitchen.washer,construction.tools.light,3897
9,apparel.shoes.slipons,construction.tools.light,3680
